# Mail Assist
Eerst pakketjes installeren

In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 14.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.8/210.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.4/409.4 kB 11.1 MB/s eta 0:00:0000:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.5
    Uninstalling protobuf-3.19.5:
      Successfully uninstalled protobuf-3.19.5
  Attempting uninstall: google-auth
    Found existing installation: google-auth 1.26.1
    Uninstalling google-auth-1.26.1:
      Successfully uninstalled google-auth-1.26.1
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Uninstalling google-auth-oauthlib-0.4.6:
      Successfully uninstalled google-auth-oauthlib-0.4.6
  Attempting uninstall: google-auth-httplib2
    Found existing installation: google-auth-httplib2 0.1.0
    Uninstalling google-auth-httplib2-0.1.0:
      Successfully uninstalled google-auth-htt

Dan connectie maken (en authenticaten)

In [6]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import base64

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]


"""Shows basic usage of the Gmail API.
Lists the user's Gmail labels.
"""
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "client_secret_1072693218348-r2cervhc49ahoa7o9ncam7c7k9alvgnu.apps.googleusercontent.com.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

helperfunctie maken om mails te verwerken

In [18]:
import base64
from email.utils import parsedate_to_datetime

import re
from bs4 import BeautifulSoup

def clean_email(email_body):

    # Verwijder HTML-tags
    soup = BeautifulSoup(email_body, "html.parser")
    email_body = soup.get_text()

    # Verwijder handtekeningen en doorgestuurde berichten
    email_body = re.split(r'Van:|Op:|Met vriendelijke groet,', email_body)[0]

    # Verwijder URL's en e-mailadressen
    email_body = re.sub(r'http\S+|www\S+|[\w\.-]+@[\w\.-]+', '', email_body)

    # Normaliseer tekst
    #email_body = email_body.lower().strip()
    email_body = email_body.strip()
    
    email_body = re.sub(r'http\S+|www\S+', '', email_body)

    # Verwijder specifieke LinkedIn-tracking parameters
    email_body = re.sub(r'midToken=\S+', '', email_body)
    email_body = re.sub(r'midSig=\S+', '', email_body)
    email_body = re.sub(r'trk=\S+', '', email_body)
    email_body = re.sub(r'trkEmail=\S+', '', email_body)
    email_body = re.sub(r'lipi=\S+', '', email_body)

    # Verwijder overbodige spaties en regels
    email_body = re.sub(r'\s+', ' ', email_body).strip()

    return email_body


def get_message_content(service, msg_id):
    # Verkrijg de e-mail met de message_id
    message = service.users().messages().get(userId='me', id=msg_id).execute()
    
    # Haal het onderwerp, de body en de datum van de e-mail op
    subject = None
    mail_body = None
    date_sent = None
    
    # Doorloop de headers om het onderwerp, de datum en andere gegevens te vinden
    for header in message['payload']['headers']:
        if header['name'] == 'Subject':
            subject = header['value']
        if header['name'] == 'Date':
            # Parse de 'Date' header naar een datetime object
            date_sent = parsedate_to_datetime(header['value'])
    
    # Haal de body op (mogelijk meerdere delen, bijv. tekst en HTML)
    if 'parts' in message['payload']:
        for part in message['payload']['parts']:
            if part['mimeType'] == 'text/plain':
                mail_body = part['body']['data']
                # Decodeer de body van base64
                mail_body = base64.urlsafe_b64decode(mail_body).decode('utf-8')
                mail_body = clean_email(mail_body)
                mail_body = cut_off_regards(mail_body)
                mail_body = get_first_n_words(mail_body)
    
    return subject, mail_body, date_sent

def cut_off_regards(email_body):
    # Define common sign-offs
    sign_offs = [
        r'Best regards,',
        r'Kind regards,',
        r'Sincerely,',
        r'Regards,',
        r'Best,',
        r'Cheers',
    ]

    # Combine sign-offs into a single regex pattern
    sign_off_pattern = '|'.join(sign_offs)

    # Find the position of the first sign-off
    match = re.search(sign_off_pattern, email_body, re.IGNORECASE)

    if match:
        # Truncate the email body at the sign-off
        email_body = email_body[:match.start()]

    # Additional cleaning steps (e.g., removing URLs)
    email_body = re.sub(r'http\S+|www\S+', '', email_body)
    email_body = re.sub(r'\s+', ' ', email_body).strip()

    return email_body

def get_first_n_words(email_body, n=50):
    # Split the email body into words
    words = email_body.split()

    # Select the first n words
    first_n_words = ' '.join(words[:n])

    return first_n_words

Query en save to file

In [21]:
# Fetch the last 100 messages
service = build("gmail", "v1", credentials=creds)
results = service.users().messages().list(
    userId='me', 
    q='from:("joerihonnef@gmail.com")', 
    maxResults=50
).execute()

messages = results.get('messages', [])
print(f"Aantal berichten gevonden: {len(messages)}")
all_content = ""

if messages:
    for message in messages:
        msg_id = message['id']
        subject, mail_body, date = get_message_content(service, msg_id)
        if subject:
            all_content += f"Subject: {subject}\n"
        if mail_body:
            all_content += f"Sent on: {date}\n\n"
        if mail_body:
            all_content += f"Body: {mail_body}\n\n"

# Export to a file
with open("email_content.txt", "w", encoding="utf-8") as f:
    f.write(all_content)

print("Data written to email_content.txt")


Aantal berichten gevonden: 50
Data written to email_content.txt
